## Giới thiệu về bộ dữ liệu sẽ xử lý

- Bộ dữ liệu raw bao gồm 4 file `laptop_basic.csv`, `laptop_detail.csv`, `cpuben.csv` và `cpuben.csv`. Dữ liệu được cào từ website [LaptopMedia](https://laptopmedia.com/)
- File `laptop_basic.csv` chứa link thông tin của 10000 laptop.
- File `laptop_detail.csv` chứa thông tin chi tiết của 10000 laptop.
- File `cpuben.csv` chứa thông tin benchmark và thứ hạng hiệu năng của 122 mẫu CPU.
- File `gpuben.csv` chứa thông tin benchmark và thứ hạng hiệu năng của 140 mẫu GPU.
- Trong notebook này, chúng em sẽ tiến hành gộp, xử lý và làm sạch dữ liệu. Kết quả cuối cùng sẽ được lưu vào file 'clean_data.csv' 

## 1. Khai báo thư viện


In [68]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

## 2. Load dữ liệu

In [69]:
data_detail = pd.read_csv('raw\laptop_detail.csv')
print('Number of rows and columns:', data_detail.shape)

Number of rows and columns: (10000, 97)


In [70]:
data_detail.head(3)

,Unnamed: 0.1,Unnamed: 0,Product Name,Product Link,Processor,Internal memory,Video card,Display,Solid-state drive,Price,...,Storage drive heater\n,4G LTE multi carrier mobile broadband with satellite GPS,E-compass\n,Accelerometer,4x RAM slots,G-sensor,Accelerometer (G-sensor),Slot Integrated Bluetooth Earbuds,3W subwoofer,MIL-STD 810G tested
0,0,0,ASUS TUF Dash F15,https://laptopmedia.com/laptop-specs/asus-tuf-...,Intel Core i7-12650H,64GB RAM,"NVIDIA GeForce RTX 3070 (Laptop, 105W)","15.6”, Full HD (1920 x 1080), 144 Hz, IPS",2000GB SSD,$362900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,Lenovo Legion 5i 15,https://laptopmedia.com/laptop-specs/lenovo-le...,Intel Core i7-12700H,64GB RAM,"NVIDIA GeForce RTX 3060 (Laptop, 130W)","15.6”, QHD (2560 x 1440), 165 Hz, IPS",2000GB SSD,$304999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,Acer Nitro 5,https://laptopmedia.com/laptop-specs/acer-nitr...,Intel Core i7-12700H,32GB RAM,"NVIDIA GeForce RTX 3060 (Laptop, 140W)","15.6”, Full HD (1920 x 1080), 144 Hz, IPS",2000GB SSD,$181900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Làm sạch dữ liệu

### 3.1 Xóa hai cột thông tin dư đầu tiên


In [71]:
# delete the first two column
data_detail = data_detail.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)


### 3.2 Xóa các cột thông tin chứa quá nhiều dữ liệu trống


#### 3.2.1 Take a look at the data


In [72]:
print('Top 25 cột có nhiều giá trị non-null nhất:')
data_detail.count().sort_values(ascending=False)[:25]

Top 25 cột có nhiều giá trị non-null nhất:


Product Name          10000
Processor             10000
Internal memory       10000
Video card            10000
Product Link          10000
Price                 10000
GPU                    9999
Features               9999
Weight                 9999
RAM                    9998
Detail price           9997
HDD/SSD                9996
Display                9985
OS                     9975
Dimensions             9950
Solid-state drive      9866
Web camera             9237
Speakers               8977
Microphone             7443
Gifts                  6504
Body Material          4639
M.2 Slot               4561
Security Lock slot     2283
Hard drive              688
Fingerprint reader       53
dtype: int64

- Với cột 'Hard drive' các dữ liệu trống sẽ được thay thế bằng '0 GB' để dễ xử lý sau này.

In [73]:
# fill missing value in Hard drive with 0GB
data_detail['Hard drive'] = data_detail['Hard drive'].fillna('0GB')

#### 3.2.2 Xóa các cột thông tin chứa không quá 15% dữ liệu non-null trong tập dữ liệu


In [74]:
def filter_column_meaningless(df, percent=0.85):
    filtered_column = []
    for column in df.columns:
        if (df[column].notnull().sum() <= percent * df.shape[0]):
            filtered_column.append(column)
    return filtered_column


data = data_detail.drop(columns=filter_column_meaningless(data_detail))
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Product Name       10000 non-null  object
 1   Product Link       10000 non-null  object
 2   Processor          10000 non-null  object
 3   Internal memory    10000 non-null  object
 4   Video card         10000 non-null  object
 5   Display            9985 non-null   object
 6   Solid-state drive  9866 non-null   object
 7   Price              10000 non-null  object
 8   Hard drive         10000 non-null  object
 9   Detail price       9997 non-null   object
 10  GPU                9999 non-null   object
 11  HDD/SSD            9996 non-null   object
 12  RAM                9998 non-null   object
 13  OS                 9975 non-null   object
 14  Dimensions         9950 non-null   object
 15  Weight             9999 non-null   object
 16  Features           9999 non-null   object

#### 3.2.3 Chỉnh sửa thông tin và xóa đi các cột thông tin không cần thiết/ trùng lặp

- Giữ lại cột 'Solid-state-drive' và 'Hard drive' vì dữ liệu dễ xử lý và sạch hơn cột 'HDD/SSD'. Cột 'HDD/SSD' có nhiều dữ liệu không đồng nhất.

In [75]:
data['HDD/SSD'].unique()[:25]


array(['2TB SSD RAID 0', '2TB SSD RAID 1', '2TB SSD', '1TB SSD',
       '1TB SSD ', '8TB SSD', '4TB SSD', '512GB SSD',
       '512GB SSD + 1TB HDD', '256GB SSD + 1TB HDD', '2TB SSD + 2TB HDD',
       '256GB SSD', '2TB SSD ', '1TB SSD RAID 1', '128GB SSD',
       '32GB eMMC', '512GB PCIe NVMe SSD', '500GB HDD', '8TB SSD ',
       '128GB SSD + 1TB HDD', '1TB HDD', '64GB eMMC', '1TB SSD RAID 0',
       '128GB eMMC', '1TB SSD  + 2TB HDD'], dtype=object)

In [76]:
data = data.drop(columns='HDD/SSD')

- Đối với giá sản phẩm sẽ chỉ giữ lại cột 'Detail price' vì cột này có định dạng giá đúng.


In [77]:
data = data.drop(columns='Price')


- Đối với 'RAM' và 'Internal memory' sẽ giữ lại cột 'RAM' vì định dạng dữ liệu đơn giản hơn.


In [78]:
data = data.drop(columns='Internal memory')


- Giữ lại cột 'Video card' vì định dạng dữ liệu đơn giản hơn cột 'GPU'.

In [79]:
data = data.drop(columns='GPU')

- Cột 'Web camera' và 'Speakers' không có nhiều thông tin (chỉ có thông tin về tên của camera và loa) nên sẽ xóa.


In [80]:
# Some web camera values
data['Web camera'].unique()[10:25]


array(['HD with camera shutter, temporal noise reduction', 'HD or FHD',
       'HD / FHD / IR with privacy shutter, fixed focus', 'HD 720p',
       '5MP with IR', 'FHD with privacy shutter, fixed focus', '5MP',
       'HD TrueVision with temporal noise reduction', 'FHD & IR', 'HD IR',
       'FHD IR with Windows Hello', 'HD with privacy shutter',
       'FHD IR Camera for Windows Hello', 'HD + FHD', '2MP Full HD'],
      dtype=object)

In [81]:
# Some speaker values
data['Speakers'].unique()[10:25]


array(['2x 1.5W Stereo Speakers, Dolby Audio', '2 Speakers',
       '2x 2.5W Stereo Speakers', '4x Speakers, Smart Amp, Dolby Atmos',
       '2x 2W, DTS: X Ultra', 'Stereo Speakers',
       '2x 2W Speaker, Dynaudio', '2x Stereo Speakers, Acer TrueHarmony',
       '2x Speakers',
       ' 2x 2W Dual-Force Woofer + 2x 2W Tweeter, Smart Amp, Dolby Atmos',
       'Stereo Speakers, Bang & Olufsen, DTS:X Ultra',
       'Speaker System optimized by DTS Audio', '2x 2W, Dolby Audio',
       '2x 2W Speakers + 4x 2W Woofers, Dynaudio, Nahimic 3',
       '2x 1.5W, Dolby Atmos'], dtype=object)

In [82]:
data = data.drop(columns=['Web camera', 'Speakers'])


- Cuối cùng, xóa cột 'Product Link' vì không cần thiết.


In [83]:
data = data.drop(columns='Product Link')


### 3.3 Kiểm tra lại tập dữ liệu


In [84]:
data.head(10)

,Product Name,Processor,Video card,Display,Solid-state drive,Hard drive,Detail price,RAM,OS,Dimensions,Weight,Features
0,ASUS TUF Dash F15,Intel Core i7-12650H,"NVIDIA GeForce RTX 3070 (Laptop, 105W)","15.6”, Full HD (1920 x 1080), 144 Hz, IPS",2000GB SSD,0GB,$3629.00.None,64GB DDR5,Windows 11 Pro,"354 x 251 x 19.9 - 20.7 mm (13.94"" x 9.88"" x 0...",2.00 kg (4.4 lbs),Fingerprint reader
1,Lenovo Legion 5i 15,Intel Core i7-12700H,"NVIDIA GeForce RTX 3060 (Laptop, 130W)","15.6”, QHD (2560 x 1440), 165 Hz, IPS",2000GB SSD,0GB,$3049.99.None,64GB DDR5,Windows 11 Pro,"358.8 x 262.35 x 19.99 - 24.05 mm (14.13"" x 10...",2.40 kg (5.3 lbs),Fingerprint reader
2,Acer Nitro 5,Intel Core i7-12700H,"NVIDIA GeForce RTX 3060 (Laptop, 140W)","15.6”, Full HD (1920 x 1080), 144 Hz, IPS",2000GB SSD,0GB,$1819.00.None,32GB DDR5,Windows 11 Home,"360.4 x 271.1 x 25.9 - 26.9 mm (14.19"" x 10.67...",2.50 kg (5.5 lbs),Fingerprint reader
3,Acer Predator Triton 500 SE,Intel Core i7-11800H,"NVIDIA GeForce RTX 3060 (Laptop, 105W)","16.0"", WQXGA (2560 x 1600), 165 Hz, IPS",1000GB SSD,0GB,$2359.99.None,32GB DDR4,Windows 10 Pro,"358.14 x 261.62 x 19.81 mm (14.10"" x 10.30"" x ...",2.44 kg (5.4 lbs),Fingerprint reader
4,HP 15,Intel Core i5-1155G7,Intel Iris Xe Graphics G7 (80EU),"15.6”, HD (1366 x 768), TN",2000GB SSD,0GB,$1309.99.None,64GB DDR4,Windows 11 Home,"358 x 242 x 17.9 mm (14.09"" x 9.53"" x 0.70"")",1.70 kg (3.7 lbs),Web camera
5,Dell Inspiron 5425,AMD Ryzen 7 5825U,"AMD Radeon RX Vega 8 (R4000/5000, 15W)","14.0"", FHD+ (1920 x 1200), IPS",1000GB SSD,0GB,$1099.99.None,16GB DDR4,Windows 11 Pro,"314 x 227.50 x 15.70 - 18.37 mm (12.36"" x 8.96...",1.55 kg (3.4 lbs),Fingerprint reader
6,Lenovo V15 Gen 2,Intel Core i7-1165G7,Intel Iris Xe Graphics G7 (96EU),"15.6”, Full HD (1920 x 1080), IPS",1000GB SSD,0GB,$719.99.None,16GB DDR4,Windows 11 Pro,"359.2 x 235.8 x 19.9 mm (14.14"" x 9.28"" x 0.78"")",1.70 kg (3.7 lbs),Fingerprint reader
7,HP EliteBook 840 G8,Intel Core i5-1135G7,Intel Iris Xe Graphics G7 (80EU),"14.0”, Full HD (1920 x 1080), IPS",1000GB SSD,0GB,$1789.00.None,64GB DDR4,Windows 11 Pro,"323.3 x 214.6 x 17.9 mm (12.73"" x 8.45"" x 0.70"")",1.32 kg (2.9 lbs),Fingerprint reader
8,HP ENVY x360 15,Intel Core i5-1135G7,Intel Iris Xe Graphics G7 (80EU),"15.6”, Full HD (1920 x 1080), IPS",8000GB SSD,0GB,$4649.00.None,64GB DDR4,Windows 10 Pro,"358.9 x 228 x 18.4 mm (14.13"" x 8.98"" x 0.72"")",1.82 kg (4 lbs),Fingerprint reader
9,HP 15,Intel Core i7-1260P,Intel Iris Xe Graphics G7 (96EU),"15.6”, Full HD (1920 x 1080), IPS",1000GB SSD,0GB,$989.00.None,32GB DDR4,Windows 11 Pro,"358 x 242 x 17.9 mm (14.09"" x 9.53"" x 0.70"")",1.70 kg (3.7 lbs),Fingerprint reader


In [85]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Product Name       10000 non-null  object
 1   Processor          10000 non-null  object
 2   Video card         10000 non-null  object
 3   Display            9985 non-null   object
 4   Solid-state drive  9866 non-null   object
 5   Hard drive         10000 non-null  object
 6   Detail price       9997 non-null   object
 7   RAM                9998 non-null   object
 8   OS                 9975 non-null   object
 9   Dimensions         9950 non-null   object
 10  Weight             9999 non-null   object
 11  Features           9999 non-null   object
dtypes: object(12)
memory usage: 937.6+ KB


## 4. Gộp và chỉnh sửa lại dữ liệu (imputation, remove duplicates,...)


### 4.1 Gộp thông tin benchmark của CPU và GPU vào data


In [86]:
data_cpu = pd.read_csv('raw\cpuben.csv')
data_gpu = pd.read_csv('raw\gpuben.csv')


In [87]:
# Merge CPU data
data = pd.merge(data, data_cpu, left_on='Processor',
                right_on='CPU', how='left')
data.rename(columns={'Rank': 'Rank CPU'}, inplace=True)


In [88]:
data = pd.merge(data, data_gpu, left_on='Video card',
                right_on='GPU', how='left')
data.rename(columns={'Rank': 'Rank GPU'}, inplace=True)


In [89]:
data.head()


,Product Name,Processor,Video card,Display,Solid-state drive,Hard drive,Detail price,RAM,OS,Dimensions,Weight,Features,Rank CPU,CPU,Cinebench23,Rank GPU,GPU,3DMark Time Spy (G)
0,ASUS TUF Dash F15,Intel Core i7-12650H,"NVIDIA GeForce RTX 3070 (Laptop, 105W)","15.6”, Full HD (1920 x 1080), 144 Hz, IPS",2000GB SSD,0GB,$3629.00.None,64GB DDR5,Windows 11 Pro,"354 x 251 x 19.9 - 20.7 mm (13.94"" x 9.88"" x 0...",2.00 kg (4.4 lbs),Fingerprint reader,33.0,Intel Core i7-12650H,13001.0,NaN,NaN,NaN
1,Lenovo Legion 5i 15,Intel Core i7-12700H,"NVIDIA GeForce RTX 3060 (Laptop, 130W)","15.6”, QHD (2560 x 1440), 165 Hz, IPS",2000GB SSD,0GB,$3049.99.None,64GB DDR5,Windows 11 Pro,"358.8 x 262.35 x 19.99 - 24.05 mm (14.13"" x 10...",2.40 kg (5.3 lbs),Fingerprint reader,17.0,Intel Core i7-12700H,15486.0,35.0,"NVIDIA GeForce RTX 3060 (Laptop, 130W)",8841.0
2,Acer Nitro 5,Intel Core i7-12700H,"NVIDIA GeForce RTX 3060 (Laptop, 140W)","15.6”, Full HD (1920 x 1080), 144 Hz, IPS",2000GB SSD,0GB,$1819.00.None,32GB DDR5,Windows 11 Home,"360.4 x 271.1 x 25.9 - 26.9 mm (14.19"" x 10.67...",2.50 kg (5.5 lbs),Fingerprint reader,17.0,Intel Core i7-12700H,15486.0,34.0,"NVIDIA GeForce RTX 3060 (Laptop, 140W)",8864.0
3,Acer Predator Triton 500 SE,Intel Core i7-11800H,"NVIDIA GeForce RTX 3060 (Laptop, 105W)","16.0"", WQXGA (2560 x 1600), 165 Hz, IPS",1000GB SSD,0GB,$2359.99.None,32GB DDR4,Windows 10 Pro,"358.14 x 261.62 x 19.81 mm (14.10"" x 10.30"" x ...",2.44 kg (5.4 lbs),Fingerprint reader,37.0,Intel Core i7-11800H,12390.0,39.0,"NVIDIA GeForce RTX 3060 (Laptop, 105W)",8290.0
4,HP 15,Intel Core i5-1155G7,Intel Iris Xe Graphics G7 (80EU),"15.6”, HD (1366 x 768), TN",2000GB SSD,0GB,$1309.99.None,64GB DDR4,Windows 11 Home,"358 x 242 x 17.9 mm (14.09"" x 9.53"" x 0.70"")",1.70 kg (3.7 lbs),Web camera,NaN,NaN,NaN,115.0,Intel Iris Xe Graphics G7 (80EU),1097.0


Chúng ta có thể thấy rằng có một số CPU và GPU trong data không có dữ liệu benchmark. Vì vậy, chúng ta sẽ loại bỏ cột Processor và Video card để đảm bảo việc xóa dữ liệu trống về sau.


In [90]:
# Delete Processor and Video card columns
data = data.drop(columns=['Processor', 'Video card'])


In [91]:
data.head()


,Product Name,Display,Solid-state drive,Hard drive,Detail price,RAM,OS,Dimensions,Weight,Features,Rank CPU,CPU,Cinebench23,Rank GPU,GPU,3DMark Time Spy (G)
0,ASUS TUF Dash F15,"15.6”, Full HD (1920 x 1080), 144 Hz, IPS",2000GB SSD,0GB,$3629.00.None,64GB DDR5,Windows 11 Pro,"354 x 251 x 19.9 - 20.7 mm (13.94"" x 9.88"" x 0...",2.00 kg (4.4 lbs),Fingerprint reader,33.0,Intel Core i7-12650H,13001.0,NaN,NaN,NaN
1,Lenovo Legion 5i 15,"15.6”, QHD (2560 x 1440), 165 Hz, IPS",2000GB SSD,0GB,$3049.99.None,64GB DDR5,Windows 11 Pro,"358.8 x 262.35 x 19.99 - 24.05 mm (14.13"" x 10...",2.40 kg (5.3 lbs),Fingerprint reader,17.0,Intel Core i7-12700H,15486.0,35.0,"NVIDIA GeForce RTX 3060 (Laptop, 130W)",8841.0
2,Acer Nitro 5,"15.6”, Full HD (1920 x 1080), 144 Hz, IPS",2000GB SSD,0GB,$1819.00.None,32GB DDR5,Windows 11 Home,"360.4 x 271.1 x 25.9 - 26.9 mm (14.19"" x 10.67...",2.50 kg (5.5 lbs),Fingerprint reader,17.0,Intel Core i7-12700H,15486.0,34.0,"NVIDIA GeForce RTX 3060 (Laptop, 140W)",8864.0
3,Acer Predator Triton 500 SE,"16.0"", WQXGA (2560 x 1600), 165 Hz, IPS",1000GB SSD,0GB,$2359.99.None,32GB DDR4,Windows 10 Pro,"358.14 x 261.62 x 19.81 mm (14.10"" x 10.30"" x ...",2.44 kg (5.4 lbs),Fingerprint reader,37.0,Intel Core i7-11800H,12390.0,39.0,"NVIDIA GeForce RTX 3060 (Laptop, 105W)",8290.0
4,HP 15,"15.6”, HD (1366 x 768), TN",2000GB SSD,0GB,$1309.99.None,64GB DDR4,Windows 11 Home,"358 x 242 x 17.9 mm (14.09"" x 9.53"" x 0.70"")",1.70 kg (3.7 lbs),Web camera,NaN,NaN,NaN,115.0,Intel Iris Xe Graphics G7 (80EU),1097.0


### 4.2 Xóa dữ liệu trống và trùng lặp


In [92]:
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)
data = data.reset_index(drop=True)


In [93]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7124 entries, 0 to 7123
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Product Name         7124 non-null   object 
 1   Display              7124 non-null   object 
 2   Solid-state drive    7124 non-null   object 
 3   Hard drive           7124 non-null   object 
 4   Detail price         7124 non-null   object 
 5   RAM                  7124 non-null   object 
 6   OS                   7124 non-null   object 
 7   Dimensions           7124 non-null   object 
 8   Weight               7124 non-null   object 
 9   Features             7124 non-null   object 
 10  Rank CPU             7124 non-null   float64
 11  CPU                  7124 non-null   object 
 12  Cinebench23          7124 non-null   float64
 13  Rank GPU             7124 non-null   float64
 14  GPU                  7124 non-null   object 
 15  3DMark Time Spy (G)  7124 non-null   f

In [94]:
data.head()


,Product Name,Display,Solid-state drive,Hard drive,Detail price,RAM,OS,Dimensions,Weight,Features,Rank CPU,CPU,Cinebench23,Rank GPU,GPU,3DMark Time Spy (G)
0,Lenovo Legion 5i 15,"15.6”, QHD (2560 x 1440), 165 Hz, IPS",2000GB SSD,0GB,$3049.99.None,64GB DDR5,Windows 11 Pro,"358.8 x 262.35 x 19.99 - 24.05 mm (14.13"" x 10...",2.40 kg (5.3 lbs),Fingerprint reader,17.0,Intel Core i7-12700H,15486.0,35.0,"NVIDIA GeForce RTX 3060 (Laptop, 130W)",8841.0
1,Acer Nitro 5,"15.6”, Full HD (1920 x 1080), 144 Hz, IPS",2000GB SSD,0GB,$1819.00.None,32GB DDR5,Windows 11 Home,"360.4 x 271.1 x 25.9 - 26.9 mm (14.19"" x 10.67...",2.50 kg (5.5 lbs),Fingerprint reader,17.0,Intel Core i7-12700H,15486.0,34.0,"NVIDIA GeForce RTX 3060 (Laptop, 140W)",8864.0
2,Acer Predator Triton 500 SE,"16.0"", WQXGA (2560 x 1600), 165 Hz, IPS",1000GB SSD,0GB,$2359.99.None,32GB DDR4,Windows 10 Pro,"358.14 x 261.62 x 19.81 mm (14.10"" x 10.30"" x ...",2.44 kg (5.4 lbs),Fingerprint reader,37.0,Intel Core i7-11800H,12390.0,39.0,"NVIDIA GeForce RTX 3060 (Laptop, 105W)",8290.0
3,Dell Inspiron 5425,"14.0"", FHD+ (1920 x 1200), IPS",1000GB SSD,0GB,$1099.99.None,16GB DDR4,Windows 11 Pro,"314 x 227.50 x 15.70 - 18.37 mm (12.36"" x 8.96...",1.55 kg (3.4 lbs),Fingerprint reader,52.0,AMD Ryzen 7 5825U,10523.0,114.0,"AMD Radeon RX Vega 8 (R4000/5000, 15W)",1129.0
4,Lenovo V15 Gen 2,"15.6”, Full HD (1920 x 1080), IPS",1000GB SSD,0GB,$719.99.None,16GB DDR4,Windows 11 Pro,"359.2 x 235.8 x 19.9 mm (14.14"" x 9.28"" x 0.78"")",1.70 kg (3.7 lbs),Fingerprint reader,99.0,Intel Core i7-1165G7,5125.0,107.0,Intel Iris Xe Graphics G7 (96EU),1435.0


### 4.3 Chỉnh sửa lại các cột dữ liệu


In [95]:
# create new dataframe
clean_df = pd.DataFrame()


#### 4.3.1 Chỉnh sửa thông tin giá

- **Price** : giá laptop (đơn vị: USD) là thông tin được sử dụng để dự đoán.

In [96]:
clean_df['price'] = data['Detail price'].str.extract(r'\$(\d+.\d+)')
clean_df['price'] = clean_df['price'].astype('float32')


#### 4.3.2 Chỉnh sửa thông tin hãng sản xuất

- **Brand** : Laptop brand
- For example: The laptop with name 'ASUS TUF Dash F15' will have 'ASUS' brand.


In [97]:
clean_df['brand'] = data['Product Name'].str.upper().str.split().str[0]
clean_df['brand'].unique()


array(['LENOVO', 'ACER', 'DELL', 'HP', 'ASUS', 'MSI', 'LG', 'MICROSOFT',
       'AORUS', 'ALIENWARE', 'RAZER', 'PANASONIC', 'SAMSUNG', 'DYNABOOK',
       'GIGABYTE', 'ENVY'], dtype=object)

#### 4.3.3 Chỉnh sửa thông tin RAM

- **RAM** : Laptop RAM
- The ram will be in the form \\{ram}{some whitespace}GB so we use this regex r'(\d+)\s\*GB' to extract it.


In [98]:
clean_df['ram'] = data['RAM'].str.upper().str.extract(r'(\d+)\s*GB')
clean_df['ram'] = clean_df['ram'].astype('float32')
clean_df['ram'].unique()


array([ 64.,  32.,  16.,  12.,   8.,  24.,  40.,   4.,  20.,  36.,  48.,
       128.,  18.], dtype=float32)

In [99]:

clean_df['ram_type'] = data['RAM'].str.upper().str.extract(r'(\d+)\s*GB')
# get text after GB and before , (comma) or nothing
clean_df['ram_type'] = data['RAM'].str.upper().str.extract(r'GB\s*(\w*)\,?')
# print numver of non-null value
clean_df['ram_type'].unique()


array(['DDR5', 'DDR4', 'LPDDR4X', 'LPDDR5', 'RAM', 'LPDDR4', 'LPDDR3'],
      dtype=object)

In [100]:
clean_df['ram_type'].value_counts()

ram_type
DDR4       5513
DDR5        912
LPDDR5      338
LPDDR4X     327
LPDDR4       31
LPDDR3        2
RAM           1
Name: count, dtype: int64

Ta thấy không có loại ram nào có loại là RAM, điều này có thể vì dữ liệu cào không sạch, ta thấy số record có 'ram_type' là 'RAM' là 1. Vì vậy, ta sẽ xóa các record này.

In [101]:
# delete rows with ram_type is 'RAM'
clean_df = clean_df[clean_df['ram_type'] != 'RAM']

#### 4.3.4 Chỉnh sửa thông tin về màn hình laptop

- **Display Resolution** : Laptop display resolution
- Từ các dòng dữ liệu: "15.6”, Full HD (1920 x 1080), 144 Hz, IPS" trong cột 'Display' ta trích xuất ra thêm các thuộc tính: 
    - **'display_size'** : Kích thước màn hình laptop (inch)
    - **'display_type'** : Loại màn hình laptop (HD, Full HD, 4K, OLED, ...)
    - **'pixels'** : Độ phân giải màn hình laptop (pixels)
    - **'screen_rate'** : Tần số làm tươi màn hình laptop (Hz)
    - **'screen_panel'** : Loại panel màn hình laptop (IPS, TN, OLED, ...)


In [102]:
# tìm kiếm kích thước màn hình
clean_df['display_size'] = data['Display'].str.extract(r'^(\d+\.?\d+)')
clean_df['display_size'] = clean_df['display_size'].astype('float32')
clean_df['display_type'] = data['Display'].str.extract(r'[,](.*?)[(]')
clean_df['resolution'] = data['Display'].str.extract(r'\((.*?)\)')

# create column  for resolution

clean_df['pwidth'] = clean_df['resolution'].str.split('x').str[0]
clean_df['pheight'] = clean_df['resolution'].str.split('x').str[1]
# Convert the width and height columns to integers
clean_df['pwidth'] = clean_df['pwidth'].astype(int)
clean_df['pheight'] = clean_df['pheight'].astype(int)

# Calculate the product of width and height to get the total number of pixels
clean_df['pixels'] = clean_df['pwidth'] * clean_df['pheight']
clean_df['pixels'] = clean_df['pixels'].astype('float32')

# delete resolution column
clean_df.drop(columns='resolution', inplace=True)
clean_df.drop(columns=['pwidth', 'pheight'], inplace=True)

clean_df['screen_rate'] = data['Display'].str.extract(r'[)][,](.*?)[H]')

clean_df['screen_panel'] = data['Display'].str.split(',').str[-1]
# remove space at the beginning and end of the string
clean_df['screen_panel'] = clean_df['screen_panel'].str.strip()


In [103]:
# print unique value of screen_panel
clean_df['screen_panel'].value_counts()

screen_panel
IPS                         5801
TN                          1145
OLED                         173
PixelSense (2256 x 1504)       2
Full HD (1920 x 1080)          2
Name: count, dtype: int64

In [104]:
clean_df['screen_panel'].unique()

array(['IPS', 'TN', 'OLED', 'PixelSense (2256 x 1504)',
       'Full HD (1920 x 1080)'], dtype=object)

In [105]:
# delete rows with screen_panel is 'PixelSense (2256 x 1504)'
clean_df = clean_df[clean_df['screen_panel'] != 'PixelSense (2256 x 1504)']
# delete rows with screen_panel is 'Full HD (1920 x 1080)'
clean_df = clean_df[clean_df['screen_panel'] != 'Full HD (1920 x 1080)']


Ta thấy một số record PixelSense (2256 x 1504) Full HD (1920 x 1080) ở đây, có nghĩa là những dòng này trong dữ liệu gốc không có dữ liệu về tấm nên, do đó ta xóa các record này đi

Đối với tần số quét của laptop, thông thường mặc định các màn hình đều có tần số quét là 60Hz, nếu tần số quét cao hơn, các nhà sản xuất sẽ đưa thông tin tần số quét vào thông tin sản phẩm. Vì vậy, ta sẽ thay dữ liệu trống với tần số quét là 60Hz.

In [106]:
# fill na in screen_rate by 0
clean_df['screen_rate'].fillna('60', inplace=True)
# trim space in screen_rate
clean_df['screen_rate'] = clean_df['screen_rate'].str.strip()
# convert screen_rate to float
clean_df['screen_rate'] = clean_df['screen_rate'].astype('float32')
clean_df['screen_rate'].unique()


array([165., 144.,  60., 240., 120., 300.,  90., 360.], dtype=float32)

In [107]:
clean_df['display_type'].unique()

array([' QHD ', ' Full HD ', ' WQXGA ', ' FHD+ ', ' HD+ ', ' HD ',
       '  QHD ', ' WQHD ', ' WQUXGA ', ' QHD+ ', ' WUXGA ', ' 4K UHD ',
       ' 2.5K ', ' 2.2K ', ' WQUXGA\xa0', ' 2.8K ', ' FHD ',
       ' PixelSense ', ' 3.5K ', ' UHD+ ', ' 3K ', ' 3K2K ', ' WQXGA\xa0',
       ' 2K ', ' 3.2K ', ' WUXGA+ ', ' WQHD+ ', ' 4K UHD+ '], dtype=object)

Chúng ta có thể thấy cột display_type chưa được loại bỏ khoảng trắng và ký tự đặc biệt. Vì vậy, chúng ta sẽ loại bỏ khoảng trắng và ký tự đặc biệt trong cột này. \xa0 = space

In [108]:
# trim space before and after string in display_type
clean_df['display_type'] = clean_df['display_type'].str.strip()
# change value 'Full HD' to 'FHD'
clean_df['display_type'] = clean_df['display_type'].str.replace('Full HD', 'FHD')

In [109]:
clean_df['display_type'].unique()

array(['QHD', 'FHD', 'WQXGA', 'FHD+', 'HD+', 'HD', 'WQHD', 'WQUXGA',
       'QHD+', 'WUXGA', '4K UHD', '2.5K', '2.2K', '2.8K', 'PixelSense',
       '3.5K', 'UHD+', '3K', '3K2K', '2K', '3.2K', 'WUXGA+', 'WQHD+',
       '4K UHD+'], dtype=object)

#### 4.3.5 Chỉnh sửa thông tin về kích thước laptop

- **Dimmension** : 358.9 x 228 x 18.4 mm -> kích thước của laptop bao gồm chiều dài, chiều rộng và độ dày (mm)


In [110]:
dimention = data['Dimensions'].str.findall(r'(\d+\.?\d+)')
# clean_df['length'] the first number in dimention
clean_df['length'] = dimention.str[0]
# clean_df['width'] the second number in dimention
clean_df['width'] = dimention.str[1]
# clean_df['height'] the third number in dimention
clean_df['height'] = dimention.str[2]


In [111]:
# convert to float
clean_df['length'] = clean_df['length'].astype('float32')
clean_df['width'] = clean_df['width'].astype('float32')
clean_df['height'] = clean_df['height'].astype('float32')


#### 4.3.6 Chỉnh sửa thông tin về hãng CPU và GPU

- **Processor** : CPU of laptop


In [112]:
clean_df['cpu_brand'] = data['CPU'].str.split().str[0]


In [113]:
clean_df['cpu_brand'].unique()


array(['Intel', 'AMD', 'Snapdragon'], dtype=object)

- **GPU** : Laptop graphic card


In [114]:
clean_df['gpu_brand'] = data['GPU'].str.upper().str.split().str[0]


In [115]:
clean_df['gpu_brand'].unique()


array(['NVIDIA', 'AMD', 'INTEL', 'QUALCOMM'], dtype=object)

#### 4.3.7 Chỉnh sửa thông tin về bộ nhớ trong SSD và HDD

- **Bộ nhớ trong HDD**


In [116]:
# get the number before GB in data['Hard drive']
clean_df['hdd'] = data['Hard drive'].str.extract(r'(\d+)\s*GB')
clean_df['hdd'] = clean_df['hdd'].astype('float32')
clean_df['hdd'].unique()


array([   0., 1000., 2000.,  500.], dtype=float32)

- **Bộ nhớ trong SSD**


In [117]:
clean_df['ssd'] = data['Solid-state drive'].str.extract(r'(\d+)\s*GB')
clean_df['ssd'] = clean_df['ssd'].astype('float32')
clean_df['ssd'].unique()


array([ 2000.,  1000.,  8000.,  4000.,   512.,   256.,   128.,    64.,
         516.,    32.,  7600.,  3000.,   120.,   250., 16000.,    16.],
      dtype=float32)

#### 4.3.8 Chỉnh sửa thông tin về hệ điều hành

- **OS**


In [118]:
clean_df['OS'] = data['OS'].str.extract(
    r'(Windows|Linux|No OS|Mac OS|Chrome OS)')


#### 4.3.9 Chỉnh sửa thông tin về cân nặng laptop

- **Weights**


In [119]:
clean_df['weight'] = data['Weight'].str.extract(r'^(\d+\.?\d+)')
clean_df['weight'] = clean_df['weight'].astype('float32')


#### 4.3.10 Chỉnh sửa thông tin về xếp hạng và điểm hiệu năng của CPU và GPU

- CPU/ GPU benchmark


In [120]:
clean_df['cpu_point'] = data['Cinebench23']
clean_df['gpu_point'] = data['3DMark Time Spy (G)']

# convert to float
clean_df['cpu_point'] = clean_df['cpu_point'].astype('float32')
clean_df['gpu_point'] = clean_df['gpu_point'].astype('float32')


#### 4.3.11 Chỉnh sửa thông tin về các chức năng khác của laptop

- **Features**


In [121]:
clean_df['features'] = data['Features']


In [122]:
clean_df['features'].unique()


array(['Fingerprint reader', 'Web camera', 'Backlit keyboard',
       'Microphone'], dtype=object)

In [123]:
# check data['Features'] have 'Fingerprint reader' or not, if yes let value = 1, else 0
clean_df['fingerprint_reader'] = data['Features'].str.contains(
    'Fingerprint reader')
clean_df['web_camera'] = data['Features'].str.contains('Web camera')
clean_df['backlit_keyboard'] = data['Features'].str.contains(
    'Backlit keyboard')
clean_df['microphone'] = data['Features'].str.contains('Microphone')


In [124]:
clean_df['microphone'].value_counts()


microphone
False    7114
True        5
Name: count, dtype: int64

In [125]:
clean_df['backlit_keyboard'].value_counts()


backlit_keyboard
False    7113
True        6
Name: count, dtype: int64

In [126]:
clean_df['web_camera'].value_counts()


web_camera
False    5643
True     1476
Name: count, dtype: int64

In [127]:
clean_df['fingerprint_reader'].value_counts()


fingerprint_reader
True     5632
False    1487
Name: count, dtype: int64

Vì dữ liệu của cột 'microphone' và 'keyboard_backlight' quá mất cân bằng. Lí do có thể vì lỗi khi cào dữ liệu hoặc website không hiển thị đầy đủ. Do đó, chúng ta sẽ xóa 2 cột này.


In [128]:
clean_df.drop(columns='microphone', inplace=True)
clean_df.drop(columns='backlit_keyboard', inplace=True)
# delete features column
clean_df.drop(columns='features', inplace=True)


Thay dữ liệu bool bằng 0 và 1

In [129]:
clean_df['fingerprint_reader'] = clean_df['fingerprint_reader'].replace([False, True], [0, 1])
clean_df['web_camera'] = clean_df['web_camera'].replace([False, True], [0, 1])
# convert to float
clean_df['fingerprint_reader'] = clean_df['fingerprint_reader'].astype('float32')
clean_df['web_camera'] = clean_df['web_camera'].astype('float32')

#### 4.3.12 Chỉnh sửa các cột dữ liệu dạng string thành uppercase

In [130]:
# conver brand, ram_type, display_type, screen_panel, cpu_brand, gpu_brand, OS to uppercase
clean_df['brand'] = clean_df['brand'].str.upper()
clean_df['ram_type'] = clean_df['ram_type'].str.upper()
clean_df['display_type'] = clean_df['display_type'].str.upper()
clean_df['screen_panel'] = clean_df['screen_panel'].str.upper()
clean_df['cpu_brand'] = clean_df['cpu_brand'].str.upper()
clean_df['gpu_brand'] = clean_df['gpu_brand'].str.upper()
clean_df['OS'] = clean_df['OS'].str.upper()


#### 4.3.13 Xóa dữ liệu trống và trùng lặp một lần nữa

In [131]:
clean_df.dropna(inplace=True)
clean_df.drop_duplicates(inplace=True)
# reset index column
clean_df.reset_index(drop=True, inplace=True)


## 5. Kiểm tra lại và lưu dữ liệu

In [132]:
clean_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6607 entries, 0 to 6606
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   price               6607 non-null   float32
 1   brand               6607 non-null   object 
 2   ram                 6607 non-null   float32
 3   ram_type            6607 non-null   object 
 4   display_size        6607 non-null   float32
 5   display_type        6607 non-null   object 
 6   pixels              6607 non-null   float32
 7   screen_rate         6607 non-null   float32
 8   screen_panel        6607 non-null   object 
 9   length              6607 non-null   float32
 10  width               6607 non-null   float32
 11  height              6607 non-null   float32
 12  cpu_brand           6607 non-null   object 
 13  gpu_brand           6607 non-null   object 
 14  hdd                 6607 non-null   float32
 15  ssd                 6607 non-null   float32
 16  OS    

In [133]:
clean_df.head()


,price,brand,ram,ram_type,display_size,display_type,pixels,screen_rate,screen_panel,length,...,cpu_brand,gpu_brand,hdd,ssd,OS,weight,cpu_point,gpu_point,fingerprint_reader,web_camera
0,3049.98999,LENOVO,64.0,DDR5,15.6,QHD,3686400.0,165.0,IPS,358.799988,...,INTEL,NVIDIA,0.0,2000.0,WINDOWS,2.40,15486.0,8841.0,1.0,0.0
1,1819.00000,ACER,32.0,DDR5,15.6,FHD,2073600.0,144.0,IPS,360.399994,...,INTEL,NVIDIA,0.0,2000.0,WINDOWS,2.50,15486.0,8864.0,1.0,0.0
2,2359.98999,ACER,32.0,DDR4,16.0,WQXGA,4096000.0,165.0,IPS,358.140015,...,INTEL,NVIDIA,0.0,1000.0,WINDOWS,2.44,12390.0,8290.0,1.0,0.0
3,1099.98999,DELL,16.0,DDR4,14.0,FHD+,2304000.0,60.0,IPS,314.000000,...,AMD,AMD,0.0,1000.0,WINDOWS,1.55,10523.0,1129.0,1.0,0.0
4,719.98999,LENOVO,16.0,DDR4,15.6,FHD,2073600.0,60.0,IPS,359.200012,...,INTEL,INTEL,0.0,1000.0,WINDOWS,1.70,5125.0,1435.0,1.0,0.0


In [134]:
clean_df.to_csv('clean_data.csv', index=False)
